In [1]:
pip install -U numpy

     |████████████████████████████████| 14.5 MB 1.7 MB/s eta 0:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.18.5
    Uninstalling numpy-1.18.5:
      Successfully uninstalled numpy-1.18.5
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

tensorflow-gpu 2.3.1 requires numpy<1.19.0,>=1.16.0, but you'll have numpy 1.19.4 which is incompatible.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import sys, os, time
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.utils import to_categorical

def top_k_accuracy(y_true, y_pred, k=1):
    argsorted_y = np.argsort(y_pred)[:,-k:]
    return np.any(argsorted_y.T == y_true.argmax(axis=1), axis=0).mean()

In [3]:
from tensorflow.keras.applications.xception import Xception, preprocess_input as pp_xception
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input as pp_vgg16
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input as pp_vgg19
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input as pp_inceptionv3
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input as pp_mobilenetv2
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input as pp_resnet50
from tensorflow.keras.applications.resnet_v2 import ResNet50V2, preprocess_input as pp_resnet50v2
from tensorflow.keras.applications.resnet_v2 import ResNet101V2, preprocess_input as pp_resnet101v2
from tensorflow.keras.applications.resnet_v2 import ResNet152V2, preprocess_input as pp_resnet152v2
from tensorflow.keras.applications.densenet import DenseNet121, preprocess_input as pp_densenet121
from tensorflow.keras.applications.densenet import DenseNet169, preprocess_input as pp_densenet169
from tensorflow.keras.applications.densenet import DenseNet201, preprocess_input as pp_densenet201
from tensorflow.keras.applications.nasnet import NASNetMobile, preprocess_input as pp_nasnetmobile
from tensorflow.keras.applications.efficientnet import EfficientNetB0, preprocess_input as pp_efficientnetb0
from keras import backend as K

In [4]:
K.clear_session()
gpu_options = tf.compat.v1.GPUOptions(allow_growth=True)
session = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))
config = tf.compat.v1.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 1
session = tf.compat.v1.Session(config=config)
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

# First 5000 images

## Load Data

In [5]:
models = [
    [Xception, 'xception', pp_xception],
    [VGG16, 'vgg16', pp_vgg16],
    [VGG19, 'vgg19', pp_vgg19],
    [InceptionV3, 'inceptionv3', pp_inceptionv3],
    [MobileNetV2, 'mobilenetv2', pp_mobilenetv2],
    [ResNet50V2, 'resnet50v2', pp_resnet50v2],
    [ResNet101V2, 'resnet101v2', pp_resnet101v2],
    [ResNet152V2, 'resnet152v2', pp_resnet152v2],
    [DenseNet121, 'densenet121', pp_densenet121],
    [DenseNet169, 'densenet169', pp_densenet169],
    [DenseNet201, 'densenet201', pp_densenet201],
    [NASNetMobile, 'nasnetmobile', pp_nasnetmobile],
    [EfficientNetB0, 'efficientnetb0', pp_efficientnetb0]
]

In [ ]:
x_val5 = np.load("data/x_val5.npy")
x_val5_2 = np.load("data/x_val5_2.npy")
y_val5 = np.load("data/y_val5.npy")
y_val5_one_hot = to_categorical(y_val5, 5000) # one-hot encoding


for i in range(13):

    gpu_options = tf.compat.v1.GPUOptions(allow_growth=True)
    session = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))
    config = tf.compat.v1.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = 1
    session = tf.compat.v1.Session(config=config)
    os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
    os.environ["CUDA_VISIBLE_DEVICES"]="0"
    
    
    modelChosen = models[i][0]
    modelName = models[i][1]
    preprocess = models[i][2]

    fileName  = '.keras/models/' + modelName +'.h5'
    model = modelChosen(weights = fileName)

    if modelName not in ['xception', 'inceptionv3']:  # these models require 299 x 299
        x_val = preprocess(x_val5) 
    else:
        x_val = preprocess(x_val5_2)
    
    y_pred = model.predict(x_val, verbose=1)
    
    print("Top 1 accuracy of", modelName + ": ", top_k_accuracy(y_val5_one_hot, y_pred, k=1))
    print("Top 5 accuracy of", modelName + ": ", top_k_accuracy(y_val5_one_hot, y_pred, k=5))
    print()
    
    K.clear_session()

# First 1000 images

## Load Data

In [19]:
# change the 3 variables for testing models
modelChosen = InceptionV3
modelName = 'inceptionv3'
preprocess = pp_inceptionv3

fileName  = '.keras/models/' + modelName +'.h5'
model = modelChosen(weights = fileName)

if modelName not in ['xception', 'inceptionv3']:  # these models require 299 x 299
    x_val1 = np.load("data/x_val1.npy") 
else:
    x_val1 = np.load("data/x_val1_2.npy")
    
y_val1 = np.load("data/y_val1.npy")
y_val1_one_hot = to_categorical(y_val1, 1000) # convert y_val to an array with binary values and columns equal to number of classes (one-hot encoding)

In [20]:
x_val = preprocess(x_val1)
y_pred = model.predict(x_val, verbose=1)

32/32 [==============================] - 4s 112ms/step


In [16]:
print("Top 5 accuracy of", modelName + ":", top_k_accuracy(y_val_one_hot, y_pred, k=5))
print("Top 1 accuracy of", modelName + ":", top_k_accuracy(y_val_one_hot, y_pred, k=1))

Top 5 accuracy of efficientnetb0: 0.924
Top 1 accuracy of efficientnetb0: 0.741
